pdf转txt

In [ ]:
import PyPDF2


def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text


pdf_path = 'data/rag/供电规则.pdf'  # Replace with your PDF file path
text = extract_text_from_pdf(pdf_path).replace('\n', '')
# Save the extracted text to a file
with open('data/extracted_text.txt', 'w', encoding='utf-8') as f:
    f.write(text)

#简单openai

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key='123',
    base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
)


class AuthenticationError(Exception):
    """Custom exception for authentication errors."""

    pass


def ask(question: str):
    """Ask a question to the model."""
    completion = client.chat.completions.create(
        model='qwen-max',
        messages=[
            {'role': 'system', 'content': '你是'},
            {'role': 'user', 'content': question},
        ],
    )
    return completion.choices[0].message.content


提示词模板

In [ ]:
from langchain.prompts import PromptTemplate

template = '请用一句话描述，{topic}。'
prompt = PromptTemplate(input_variables=['topic'], template=template)

formatted_prompt = prompt.format(topic='人工智能')
print(formatted_prompt)

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(
    model='qwen2.5:3b',
    messages=[
        {
            'role': 'user',
            'content': '你是谁？',
        },
    ],
)
print(response.message.content)

In [ ]:
from pydub import AudioSegment
import numpy as np
from pydub import AudioSegment

audio = AudioSegment.from_file('15.mp3')  # mp3或wav都可以


def compute_correlation(audio_path):
    # 1. 读取音频
    audio = AudioSegment.from_file(audio_path)

    if audio.channels != 2:
        raise ValueError('音频不是立体声，无法比较左右声道。')

    # 2. 拆分左右声道
    left, right = audio.split_to_mono()

    # 3. 把声道转成numpy数组（注意：raw_data是bytes）
    samples_left = np.array(left.get_array_of_samples())
    samples_right = np.array(right.get_array_of_samples())

    # 4. 相关系数计算（归一化一下）
    min_len = min(len(samples_left), len(samples_right))
    samples_left = samples_left[:min_len]
    samples_right = samples_right[:min_len]

    correlation = np.corrcoef(samples_left, samples_right)[0, 1]

    return correlation


audio_path = 'data/record/audio/15.mp3'
corr = compute_correlation(audio_path)

In [ ]:
def json_to_markdown_table(data):
    if isinstance(data, list) and all(isinstance(d, dict) for d in data):
        headers = list({key for d in data for key in d})
        lines = ['| ' + ' | '.join(headers) + ' |']
        lines.append('| ' + ' | '.join(['---'] * len(headers)) + ' |')
        for row in data:
            line = '| ' + ' | '.join(str(row.get(h, '')) for h in headers) + ' |'
            lines.append(line)
        return '\n'.join(lines)
    else:
        raise ValueError('Only supports list of dicts for table conversion.')
